In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read
from ase.neighborlist import neighbor_list
from icet import ClusterSpace
from icet import ClusterExpansion
from mchammer.observers import BinaryShortRangeOrderObserver

# read CE Data

In [2]:
simulation_folder = '/nfshome/winkelmann/CathodeSimulationResults/MMC/Li_50%/size_10_5_3/01'

ce_path = 'mixing_energy_anchorTS_lasso.ce'
# find CE

searching_folder = simulation_folder
searched_folders = []

while True:
    # Search for the file in the current directory
    test = os.path.join(searching_folder, 'program_files', ce_path)
    files = glob.glob(test)
    if files:
        ce_path = files[0]
        break
    searched_folders.append(searching_folder)
    # Move to the parent directory
    parent_dir = os.path.dirname(searching_folder)
    # If we have reached the root directory, stop searching
    if parent_dir == searching_folder:
        raise FileNotFoundError(f"Could not find 'program_files/{ce_path}' - searched:{searched_folders}")
    searching_folder = parent_dir
    
ce = ClusterExpansion.read(ce_path)
cs = ce.get_cluster_space_copy()

# Defining an Analyzer for one sim

In [ ]:
def analyze(xdatcar_path):
    # Read all frames from XDATCAR
    # The ':' tells ASE to read all frames
    structures = read(xdatcar_path, format='vasp-xdatcar', index=':')
    print(f"Loaded {len(structures)} structures from XDATCAR")
    
    #getting cutoffs 
    i,j,d = neighbor_list('ijd', structures[0], cutoff=8.1)
    unique_distances = np.unique(np.round(d, decimals=2))
    unique_distances.sort()
    cutoffs = [d + 0.1 for d in unique_distances]
    
    species = ['Li', 'X']
    
    sro_parameters = []

    # Setting up Observer
    sro = BinaryShortRangeOrderObserver(
        cluster_space=cs,
        structure=structures[0],
        radius=8.09 ) 
    
    # Process each structure-snapshot
    for i, structure in enumerate(structures):
        sro_parameters.append(sro.get_observable(structure=structure))
    
    # Plot SRO parameter evolution to identify phase transitions
    plt.figure(figsize=(12, 8))

    # Plot SRO parameters for each shell
    for i in range(n_shells):
        plt.subplot(n_shells, 1, i+1)
        plt.plot(sro_parameters[:, i], label=f'Shell {i+1} ({cutoffs[i]:.2f} Å)')
        plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
        plt.ylabel(f'SRO Parameter')
        plt.legend()
        
        # Highlight regions of significant change (potential phase transitions)
        # Calculate rolling average of derivative to find rapid changes
        window = 5  # Window size for smoothing
        if len(sro_parameters) > window*2:
            smoothed = np.convolve(sro_parameters[:, i], 
                                np.ones(window)/window, mode='valid')
            derivative = np.gradient(smoothed)
            
            # Find points where derivative exceeds threshold (potential transitions)
            threshold = 3 * np.std(derivative)
            transition_points = np.where(np.abs(derivative) > threshold)[0]
            
            # Add vertical lines at potential transition points
            for point in transition_points:
                plt.axvline(x=point+window//2, color='g', alpha=0.3)
    plt.xlabel('MC Step')
    plt.tight_layout()
    plt.savefig(xdatcar_path.replace('.xdatcar', '_sro_over_MC-step.png'), dpi=300)
    
    # sro vs MC-step
    plt.figure(figsize=(10, 6))

    # Create scatter plot of SRO vs MC step
    for i in range(n_shells):
        plt.scatter(range(n_frames), sro_parameters[:, i], 
                label=f'Shell {i+1} ({cutoffs[i]:.2f} Å)', alpha=0.5)
    
    # Calculate variance of SRO as function of MC step
    # (variance peaks at phase transitions)
    mc_steps = np.unique(temperatures)
    sro_variances = np.zeros((len(mc_steps), n_shells))

    for i, step in enumerate(mc_steps):
        step_indices = [j for j, t in enumerate(temperatures) if t == temp]
        for shell in range(n_shells):
            sro_variances[i, shell] = np.var([sro_parameters[j, shell] for j in temp_indices])
    
    # Plot variance to identify phase transition temperature
    plt.figure(figsize=(10, 6))
    for shell in range(n_shells):
        plt.plot(mc_steps, sro_variances[:, shell], 
                 marker='o', label=f'Shell {shell+1}')
    
    
    plt.xlabel('MC Step')
    plt.ylabel('SRO Parameter')
    plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    plt.grid(True)
    plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    plt.legend()
    plt.title('SRO Parameter vs MC Step')
    plt.grid(True, alpha=0.3)
    plt.savefig(xdatcar_path.replace('.xdatcar', '_sro_derivate_over_mc_step.png'), dpi=300)
    plt.show()
    
    return sro_parameters

    

# Iterating over sims

In [18]:
file_name = '*.XDATCAR'
file_path = glob.glob(os.path.join(simulation_folder, file_name))
analyze(file_path[0])

Loaded 42 structures from XDATCAR


RuntimeError: Failed to find site by position (findLatticeSiteByPosition).
Try increasing symprec or position_tolerance.
position: 2.8429 6.56539 2.35761
scaled position: 2.5 0.5 -2.5
fractional position tolerance: 2e-06